In [127]:
#import vseho mozneho
import numpy as np
import pandas as pd
import requests
import ipycanvas as ipc
from math import pi
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML

In [128]:
#ahhhh yessss
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
# -1 je neobsazeno
# daji se pouzit jakekoliv HTML barvy
player_color={
    -1:'white',
    0:'red',
    1:'blue',
    2:'green',
    3:'yellow',
    4:'orange',
    5:'magenta',
    6:'olive',
    }

In [57]:
#ziskame data ze hry
# format /game/<game_number>[/(stars|players|fleets|full|basic)]
game_number = '4905328060399616'
r = requests.get(url='http://nptriton.cqproject.net/game/'+game_number+'/stars')
data = r.json()

In [241]:
#nastavovatka
star_distance = 150
x_offset = 600
y_offset = 200

#data magic
# udelame si data frame na pandas statistiky a trochu poupravime pro statistiky
npdata = np.array(list(data.values()))
df = pd.DataFrame(columns=list(data['1'].keys()))

for i,j in enumerate(npdata):
    df.loc[i]=(list(j.values()))
    
df[['y','x']] = df[['y','x']].astype(float)
df['puid'] = df['puid'].astype(int)

#tohle je pouzitelny pro pandas
df

,uid,n,puid,v,y,x
0,1,Navi,2,0,1.7521,-1.4714
1,2,AA0,5,0,3.0511,-2.2214
2,3,Arkab,1,0,3.0511,-0.7214
3,4,Adhara,3,0,0.4530,-0.7214
4,5,Achernar,0,0,1.7521,0.0286
...,...,...,...,...,...,...
115,140,Bam,0,0,1.5611,0.0838
116,141,Toliman,0,0,1.3307,0.9291
117,142,Rye,5,0,2.4321,-0.9222
118,143,Megrez,-1,0,1.5005,1.8644


In [280]:
#Funkce, aby byly soupatka hned u vykreslovani, staci spustit jednou

#TODO vstupni informace a ne tahat z global space
def draw_galaxy(canvas,to_from):
    #nastavime kreslito na cerno a udelame cerne pozadi
    canvas.fill_style = 'black'
    canvas.fill_rect(0, 0, max_size_x, max_size_y)
  
    #nastavime pismo
    canvas.font = '11px center'
    
    #projedeme vsechny sektory a nakreslime je
    for i in to_from:    
        canvas.fill_style = player_color[i[1]]
        canvas.stroke_style= player_color[i[1]]
        canvas.shadow_color= player_color[i[1]]
        canvas.fill_text(i[0], i[3]+text_offset_x.value, i[2]+text_offset_y.value)
        canvas.fill_arc(i[3], i[2], circle_size.value, 0,pi*1.5)
        canvas.fill_style = 'white'
        canvas.fill_arc(i[3], i[2], circle_size.value/2, 0,pi*1.5)
        
    return canvas

def recalculate_sliders(to_draw):  
    to_draw = (df.drop(['uid', 'v'], axis=1))
    to_draw = to_draw.to_numpy()
    to_draw[:,2] *= slider_zoom.value
    to_draw[:,3] *= slider_zoom.value
    to_draw[:,3] += slider_offset_x.value
    to_draw[:,2] += slider_offset_y.value
    return to_draw

In [268]:
#soupatka na zoom apod. staci spustit jednou a pak soupat
slider_zoom = widgets.IntSlider(description='zoom:',min=1, max=500, value=160)
slider_offset_x = widgets.IntSlider(description='offset x:',min=-100, max=1000, value=300, step=3)
slider_offset_y = widgets.IntSlider(description='offset y:',min=-100, max=1000, value=100, step=3)

text_offset_x = widgets.IntSlider(description='text offset x:',min=-50, max=50,value=-7, step=1)
text_offset_y = widgets.IntSlider(description='text offset y:',min=-50, max=50,value=10, step=1)

circle_size = widgets.IntSlider(description='circle size:',min=2, max=45, step=1)

items_layout = widgets.Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items = [slider_zoom,slider_offset_x,slider_offset_y,text_offset_x,text_offset_y,circle_size]
box = widgets.Box(children=items, layout=box_layout)
box

Box(children=(IntSlider(value=160, description='zoom:', max=500, min=1), IntSlider(value=300, description='off…

In [288]:
#toto kresli, poustet vicekrat za sebou
max_size_x = 1000
max_size_y = 1000

canvas = ipc.Canvas(width=max_size_x, height=max_size_y)

canvas.fill_style = "white"
canvas.fill_arc(1,1, circle_size.value,0,pi*2)

to_draw = recalculate_sliders(to_draw)
draw_galaxy(canvas,to_draw)

#obrazek se da rozkliknout a zobrazit jako obrazek naplno

Canvas(height=1000, width=1000)